In [1]:
import kagglehub, os

# Download latest version
path = kagglehub.dataset_download("apollo2506/eurosat-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/apollo2506/eurosat-dataset/versions/6


In [2]:
import torchvision.transforms as transforms

from src.datasets import EuroSATDataset
from src.modules.backbones.resnet import ResNet
from src.modules.heads import FFN
from src.modules.models import EuroSATModel
from src.runner import Runner
import os


transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

loading_cfg = {
    'batch_size': 1024,
    'num_workers': 4,
}

data_cfg = {
    'root_dir': os.path.join(path, 'EuroSAT'),
    'transform': transform,
}

optim_cfg = {
    'lr': 0.001,
    'weight_decay': 1e-4,
}


model_cfg = {
    'type': EuroSATModel,
    'backbone_cfg': {
        'type': ResNet,
        'idims': 3,
        'odims': 64,
        'base_dims': 12,
        'arch': [2,2,2,2],
        'dropout': 0.2,
    },
    'head_cfg': {
        'type': FFN,
        'idims': 64,
        'odims': 10,  # EuroSAT has 10 classes
        'hidden_dims': 1024,
        'nlayers':6,
        'dropout': 0.2,
    }
}

runner = Runner(model_cfg=model_cfg, loading_cfg=loading_cfg, data_cfg=data_cfg, optim_cfg=optim_cfg, device='cuda:2', work_dir='results/eurosat')

No seed provided, using initial seed: 37885644
     Model Parameters Summary     
----------------------------------
Trainable params     | 5,857,878
Non-trainable params |         0
Total params         | 5,857,878
- Backbone params    | 1,582,668
- Head params        | 4,275,210


In [ ]:
runner.run(mode='train', val_interval=1, log_interval=1, epochs=100, start_epoch=1)

Epoch 001/100 | Iter 01/19 | [----------] |  | loss: 2.2235 | lr: 1.0000e-03